# Logistic Regression

we use logistic regression for binary classification , for those outputs are discrete ( 0 , 1 ) and it belongs to two classes in total.Logistic Regression only outputs the probablity values that features may belong the given categories ( 0 ,1 ).
## First order Optimization

In first order optimization method we will use the jacovian matrix to update the *weights* and *bias* values.
$$
\begin{align}
w_\theta &= \mathcal{V} - w_\theta^` \qquad , \mathcal{V} = \space \text{learning rate (0 to 1) } \\
b_\theta &= \mathcal{V} - b_\theta^` 
\end{align}
$$

This is a helper function to load dataset


In [2]:
def load_data(url):
    df=pd.read_csv(url,header=None);
    return ( df.iloc[:,:-1] , df.iloc[:,-1])

we are using `Sigmoid funtion` for activation . which takes input from - $\infty$ to $\infty$ and outputs from $0$ to $1$ . which is very usefull to find probability values.

$$
\begin{align}
\sigma = \frac{1}{1+e^{-x}}
\end{align}
$$

In [ ]:
def sigmoid(x):
    '''
    returns sigmoid h(x)= 1/(e^-x + 1) of the input x
    '''
    return 1/(1+np.exp(-x))

this `utility function` checks the gradient descent has converged , if converged we will stop the gradient descent training .
So convergence is checked by finding the difference between the *old* $w_\theta$ and *new* $w_\theta$ if there is'nt any much of difference then its converged.

In [ ]:
def check_for_convergence(beta_old,beta_new,tol=1e-3):
    '''
    Checks whether the coefficients have converged in the l-infinity norm.
    Returns True if they have converged, False otherwise.'''
    #calculate the change in the coefficients
    coef_change = np.abs(beta_old - beta_new)
    
    #if change hasn't reached the threshold and we have more iterations to go, keep training
    return not (np.any(coef_change>tol) )

<h3> Loss Function :</h3>	
					
$$
Cost / Loss =
 \begin{cases}
 -log(h_{\theta}(x)) & \text{if $y$ is 1} \\
 -log(1-h_{\theta}(x)) & \text{if $y$ is 0}
 \end{cases}
$$

Which can be simplified as below ,

$$
loss = \frac{1}{m}(-ylog(h_{\theta}(x) \ -(1-y)( \ log(1-h_{\theta}(x) \ ) \ ) \ ) 
$$


 let's check this figure below and see how the Cost / Loss function works .

 when $y = 1$ , we plotting $-log(h(x))$

![mtVFHjoeQkaNuKP](https://i.loli.net/2019/08/27/mtVFHjoeQkaNuKP.png)

if $h(x)$ is close to $1$ , cost is so low wherelse when $h(x)$ is close to 0 , cost is so high 

When $y=0$ , we plotting $-log(1-h(x))$

![EumjY89IzUcVh1S](https://i.loli.net/2019/08/27/EumjY89IzUcVh1S.png)

if $h(x)$ is close to $0$ , cost is so low wherelse when $h(x)$ is close to $1$ , cost is so high , make sense now !

The Deriavative of loss function is below ,

$$ \frac{\partial loss}{\partial w} = \frac{(h_\theta(x) - y)x^T}{m} $$

